### PROJECT 2 Data wrangling.

This project aims at  gathering assessing and cleaning real world data.The data used was  is the tweet archive of Twitter user @dog_rates, also known as WeRateDogs. WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a denominator of 10. The numerators, though? Almost always greater than 10. 11/10, 12/10, 13/10, etc. Why? Because "they're good dogs Brent." WeRateDogs has over 4 million followers and has received international media coverage.

The follwing steps were used to achieve the main goal(data wrangling)

# 1. Gather data

#### Directly downloaded the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

import pandas as pd

import requests

we_rate_dogs = pd.read_csv('twitter-archive-enhanced.csv',header=0)

#### Downloaded the image predictions programmatically using the requests module

response=requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

with open('image_prediction.tsv', 'wb') as file:

    file.write(response.content)

image_predictions = pd.read_csv('image_prediction.tsv', sep="\t", header=0)


#### Used the tweepy api to download additional tweet data and stored it in a txt file

The txt file then was then used to extract the additional tweet data using the json module



import json
listt = []
##### with open('tweet_json.txt', encoding='utf-8') as file:
with open('tweet_json.txt', encoding='utf-8') as file:
    for line in file.readlines():
        line = json.loads(line)
        idd = line['id']
        created_at = line['created_at']
        retweet_count = line['retweet_count']
        favorite_count = line['favorite_count']
        listt.append({'tweet_id':idd, 'favorite_count':favorite_count,'retweet_count': retweet_count, 'created_at' : created_at})

        
additional_tweets =pd.DataFrame(listt, columns=['tweet_id', 'favorite_count', 'retweet_count', 'created_at'])

# 2. ASSESS DATA

Both visual and programmatic assessment was done on all the three data sets to find * quality issues and 2 tidiness issues 

some of the methods used were

- we_rate_dogs.info()            - image_predictions.head(5)     - additional_tweets.info()

-  we_rate_dogs.head()               - image_predictions.info()

-  we_rate_dogas.describe()         - image_predictions.describe()

- we_rate_dogs.sample()            -additional_tweets.tail()

The following Quality issues were found.

1. missing values in we_rate_dogs columns (in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, expanded_urls)

2. source column in the weratedogs csv has html tags

3. Timestamp column in weratedogs has invalid datetime format

4. The ID fields, like tweet_id in the three datasets should be objects and not numeric since they are not required to perform any calculations

5. Retweets and Favorite Count: retweet_count and favorite_count should be integers, not floats.

6. rating_numerator column should be of float data type.The rating denominator is of int data type however it is preffered as float data type

7. Rating numerator  and rating denominator have anomalous values (1776, 170). 

8. In the name column, there are several values that are not dog names, like 'a', 'the', 'such', etc.

9. In image predictions csv, columns p1, p2, p3 separates two or three names with an underscore

10. Invaid date-time format in additional tweets dataframe


The following tidiness issues were found

1.The 4 different columns doggo, floofer, pupper and puppo, are all relative to the same variable that identifies the stage of dog.

2.The melted dataframe have 2 columns with the same attribute

3.The merged df1 dataset has null values in some columns.This does not follow the law that each variable forms a column


# 3. Clean Data 

After assessing the data , a copy of  each dataset was created to perform cleaning without affecting the original data

The following  methods were used 

#### drop columns with missing values more than 50%. Fill missing values in the expanded_urls column with a random url

#drop columns with missing values more than 50%.
columns = ["in_reply_to_status_id", "in_reply_to_user_id", "retweeted_status_id", "retweeted_status_user_id", "retweeted_status_timestamp"]
print(we_rate_dogs_copy.isna().mean().mul(100))
we_rate_dogs_copy.drop(columns=columns, axis=1, inplace=True)

#Fill missing values in the expanded_urls column with a random url

random_url = we_rate_dogs_copy.expanded_urls[0]
we_rate_dogs_copy['expanded_urls'].fillna(random_url, inplace=True)

#### Remove html tags in the source column to only have the url using the string split method.

def slicing(row):
    return row[1][:-4]
we_rate_dogs_copy['source'] = we_rate_dogs_copy['source'].str.split(">", 1).apply(slicing)

#### Fix the datetime format by removing the unnecessary zeros at the end of each datetime object

def time_slicing(row):
    return row[:-6]
we_rate_dogs_copy.timestamp = we_rate_dogs_copy.timestamp.apply(time_slicing)

#### change the tweet id column datatype of the three datasets from int/float to object data type using astype method.

we_rate_dogs_copy["tweet_id"] = we_rate_dogs_copy["tweet_id"].astype(str)

image_predictions_copy["tweet_id"] = image_predictions_copy["tweet_id"].astype(str)

additional_tweets_copy["tweet_id"] = additional_tweets_copy["tweet_id"].astype(str)

#### Using the astype method we will change the format to int of the columns retweet count and favorite count

def change_int(col):
    return col.astype(int)

columns =['retweet_count', 'favorite_count']
additional_tweets_copy[columns] = additional_tweets_copy[columns].apply(change_int)

#### Changing the rating numerator and rating denominator datatypes to the preferred ones

we_rate_dogs_copy.rating_numerator = we_rate_dogs_copy.rating_numerator.astype(float)
we_rate_dogs_copy.rating_denominator = we_rate_dogs_copy.rating_denominator.astype(float)

#### Drop anomalies in the dataset


we_rate_dogs_copy = we_rate_dogs_copy.drop(axis=0, index=[979])

#### Removing invalid names in the dataset eg 'like', 'such'

#drop duoplicated names
we_rate_dogs_copy = we_rate_dogs_copy.drop_duplicates(['name'])


#get valid names
def get_valid_name(row):
    if not row[0].islower():
        return row
    
we_rate_dogs_copy.name = we_rate_dogs_copy.name.apply(get_valid_name)

#drop rows that have nulls

we_rate_dogs_copy = we_rate_dogs_copy.dropna(axis = 0, how ='any')

#### In image predictions csv, we replace the underscore with an empty space  columns p1, p2, p3 using the str.replace method

columns = ['p1', 'p2', 'p3']
def replace(row):
    row.replace("_", " ")

for col in columns:
    image_predictions_copy[col] = image_predictions_copy[col].str.replace("_", " ")

#### melt columns doggo, floofer, pupper and puppo to a single column named dog_stage

we_rate_dogs_copy = pd.melt(we_rate_dogs_copy, id_vars=['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name'],
                           var_name='dog_stage')


After fixing the quality and tidy issues , I went ahead and merged the datasets.

#merge two datasets(we rate dogs and image predictions)
df1 =pd.merge(we_rate_dogs_copy, image_predictions_copy, how="left")

#remove nulls
df1 =df1.dropna(axis = 0, how ='any')

df2 = pd.merge(df1, additional_tweets_copy, how="left")

# 3. Storing Data

The merged data needed to be stored in a csv file

#finally store to our twitter archive master csv

df2.to_csv("twitter_archive_master.csv", index=False)